In [42]:
import pymongo
from sklearn.preprocessing import StandardScaler
import datetime 
import csv
import re
from collections import defaultdict
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.mov = self.db["Movie_mojo"]
        self.act = self.db["actor"]
        self.dir = self.db['director']
        self.fra = self.db['franchise']
        
def get_score(name, idt, year):
    idt = idt.split('/')
    idt = datetime.date(int(idt[2])+2000, int(idt[0]), int(idt[1]))
    ytd = datetime.timedelta(days = 365*year)

    score = 0
    for mov in name :
  #      print(mov)
        if len(name[mov]) == 4 :
            if name[mov][2] != 'NA':
                mdt = name[mov][2].split('/')
                mdt = datetime.date(int(mdt[2])+2000, int(mdt[0]), int(mdt[1]))
                if idt - mdt < ytd and datetime.timedelta(0) < idt - mdt:
                    if name[mov][3] != 'NA' :
                        score += name[mov][3]
    return score

def get_score_avg(name, idt, year):
    idt = idt.split('/')
    idt = datetime.date(int(idt[2])+2000, int(idt[0]), int(idt[1]))
    ytd = datetime.timedelta(days = 365*year)
    count = 0.
    score = 0.
    for mov in name :
  #      print(mov)
        if len(name[mov]) == 4 :
            if name[mov][2] != 'NA':
                mdt = name[mov][2].split('/')
                mdt = datetime.date(int(mdt[2])+2000, int(mdt[0]), int(mdt[1]))
                if idt - mdt < ytd and datetime.timedelta(0) < idt - mdt:
                    if name[mov][3] != 'NA' :
                        count += 1.
                        score += name[mov][3]
    return score / count


db = Database()

In [37]:
#imdb data load
with open('movie_metadata.csv', 'r') as fin:
    cin = csv.reader(fin)
    movie_imdb = [row for row in cin]
for n, i in enumerate(movie_imdb[0]):
    print(n,i)

0 color
1 director_name
2 num_critic_for_reviews
3 duration
4 director_facebook_likes
5 actor_3_facebook_likes
6 actor_2_name
7 actor_1_facebook_likes
8 gross
9 genres
10 actor_1_name
11 movie_title
12 num_voted_users
13 cast_total_facebook_likes
14 actor_3_name
15 facenumber_in_poster
16 plot_keywords
17 movie_imdb_link
18 num_user_for_reviews
19 language
20 country
21 content_rating
22 budget
23 title_year
24 actor_2_facebook_likes
25 imdb_score
26 aspect_ratio
27 movie_facebook_likes


In [98]:
arr_imdb = []
dict_imdb = {}
for row in movie_imdb[1:]:
    row[11] = re.sub('\xc2\xa0','',row[11])
    dict_imdb[row[11]] = {
        'actor' : [re.sub('[^\'\s\w]','',row[10]), re.sub('[^\'\s\w]','',row[6]), re.sub('[^\'\s\w]','',row[14])] ,
        'director' : re.sub('[^\'\s\w]','',row[1]) ,
        'budget' : row[22] ,
        'year' : row[23]
        }
    arr_imdb.append(row[11])

In [99]:
dict_act = {}
actors = db.act.find()
for act in actors:
    dict_act[act['name']] = act['movie']
dict_dir = {}
directors = db.dir.find()
for dire in directors:
    dict_dir[dire['name']] = dire['movie']
dict_fra = {}
franchise = db.fra.find()
for fra in franchise:
    dict_fra[fra['name']] = fra['movie']

In [153]:
arr_mov = [x for x in db.mov.find()]

In [181]:
arr_mov[7005]
dict_mov = {}
for mov in arr_mov:
    name = re.sub('[^\w]','',mov['Mname'])
    name = name.lower()
    dict_mov[name] = {
        'gross_kor' : mov['gross_kor'],
        'release_kor' : mov['release_kor']
    }

In [161]:
len(dict_mov)

7006

In [201]:
#def get_obj(objin):
dict_imdb = dict_imdb_actor
objin = dict_act
objlist = objin.keys()
count = 0
count1 = 0
objout = {}
for k, v in objin.items():
    k_new = re.sub('[^\w]','',k)
    k_new = k_new.lower()
    if len(v) == 0:
        try :
            for mov in dict_imdb[k_new] :
                try :
                    dict_imdb[k_new][mov] = dict_mov[mov]
                except :
                    dict_imdb[k_new][mov] = {
                        'gross_kor': 'NA', 
                        'release_kor': 'NA'
                    }
                    pass
            objout[k_new] = dict_imdb[k_new]
#            print(dict_imdb[k_new])
        except:
            objout[k_new] = {
                        'gross_kor': 'NA', 
                        'release_kor': 'NA'
                    }
            pass
    else :
        objout[k] = {}
        for 
        print(v)
        break
#objout

{u'Monsters Vs Aliens': [u'/movies/?id=monstersvsaliens.htm', u'3/27/09', u'4/23/09', 3526735], u'The Holiday': [u'/movies/?id=holiday.htm', u'12/8/06', u'12/14/06', 8217382], u'State and Main': [u'/movies/?id=stateandmain.htm', u'12/22/00', u'NA', u'NA'], u'Aloha': [u'/movies/?id=crowe2014.htm', u'5/29/15', u'NA', u'NA'], u'Leatherheads': [u'/movies/?id=leatherheads.htm', u'4/4/08', u'NA', u'NA'], u"It's Complicated": [u'/movies/?id=itscomplicated.htm', u'12/25/09', u'3/11/10', 1569953], u'The Hollars': [u'/movies/?id=thehollars.htm', u'8/26/16', u'NA', u'NA'], u'Brief Interviews with Hideous Men': [u'/movies/?id=briefinterviewswithhideousmen.htm', u'9/25/09', u'NA', u'NA'], u'Promised Land (2012)': [u'/movies/?id=promisedland2012.htm', u'12/28/12', u'NA', u'NA'], u'The Wind Rises': [u'/movies/?id=windrises.htm', u'2/21/14', u'NA', u'NA'], u'License to Wed': [u'/movies/?id=licensetowed.htm', u'7/3/07', u'NA', u'NA'], u'For Your Consideration': [u'/movies/?id=foryourconsideration.htm',

In [195]:
len(objout)

829

In [95]:
dict_imdb['Hercules']

{'actor': ['Dwayne Johnson', 'Rufus Sewell', 'Ingrid Bols Berdal'],
 'budget': '100000000',
 'director': 'Brett Ratner',
 'year': '2014'}

In [140]:
dict_imdb_actor = {}
for k, v in dict_imdb.items():
    k_new = re.sub('[^\w]','',k)
    k_new = k_new.lower()
    for actor in v['actor']:
        if actor:
            actor = re.sub('[^\w]','', actor)
            actor = actor.lower()
            try :
                if k_new not in dict_imdb_actor[actor]:
                    dict_imdb_actor[actor][k_new]={'year':v['year']}
            except:
                dict_imdb_actor[actor] = {k_new : {'year':v['year']}}
                
dict_imdb_dir = {}
for k, v in dict_imdb.items():
    k_new = re.sub('[^\w]','',k)
    k_new = k_new.lower()
    director = v['director']
    if director:
        director = re.sub('[^\w]','', director)
        director = director.lower()
        try :
            if k_new not in dict_imdb_dir[director]:
                dict_imdb_dir[director][k_new]={'year':v['year']}
        except:
            dict_imdb_dir[director] = {k_new : {'year':v['year']}}

In [141]:
dict_imdb_dir

{'sarasugarman': {'confessionsofateenagedramaqueen': {'year': '2004'}},
 'quentindupieux': {'rubber': {'year': '2010'}},
 'thomascarter': {'coachcarter': {'year': '2005'},
  'savethelastdance': {'year': '2001'},
  'whenthegamestandstall': {'year': '2014'}},
 'jobyharold': {'awake': {'year': '2007'}},
 'zachcregger': {'missmarch': {'year': '2009'}},
 'charlesbinam': {'therocketthelegendofrocketrichard': {'year': '2005'}},
 'peteryates': {'krull': {'year': '1983'}},
 'oliviermegaton': {'colombiana': {'year': '2011'},
  'taken2': {'year': '2012'},
  'taken3': {'year': '2014'}},
 'zakpenn': {'incidentatlochness': {'year': '2004'}},
 'peterrhunt': {'onhermajestyssecretservice': {'year': '1969'}},
 'microdgers': {'universalsoldierthereturn': {'year': '1999'}},
 'christophebarratier': {'thechorus': {'year': '2004'}},
 'michaelkang': {'themotel': {'year': '2005'}},
 'walterlang': {'statefair': {'year': '1945'},
  'thebluebird': {'year': '1940'}},
 'walterhill': {'bullettothehead': {'year': '20

In [89]:
dict_test = {}
for i in arr_imdb:
    s = re.sub('[^\w\d]','',i)
    try :
        dict_test[s][0] += 1
        dict_test[s].append(i)
        print(i)
    except:
        dict_test[s] = [1,i]

The Legend of Tarzan
The Twilight Saga: Breaking Dawn - Part 2
Godzilla Resurgence
Pan
Fantastic Four
The Fast and the Furious
Hercules
Total Recall
The Avengers
Dredd
Unbroken
Victor Frankenstein
The Fast and the Furious
From Hell
The Lovers
Ghostbusters
The Jungle Book
Snakes on a Plane
Point Break
King Kong
Eddie the Eagle
Chasing Liberty
Victor Frankenstein
The Island
Disturbia
Wicker Park
Conan the Barbarian
Twilight
Death at a Funeral
Glory
Forsaken
Hero
TRON: Legacy
The Illusionist
Bad Moms
Left Behind
Ben-Hur
Halloween
Aloha
Clash of the Titans
The Possession
First Blood
Dangerous Liaisons
Big Fat Liar
Teenage Mutant Ninja Turtles
RoboCop
Brothers
Casino Royale
The Alamo
The Host
Home
Poltergeist
Precious
Snatch
The Dead Zone
The Tourist
Dekalog            
Heist
Footloose
The Karate Kid
Creepshow
Syriana
Crash
Spider-Man 3
The Great Gatsby
The Claim
Skyfall
Around the World in 80 Days
Murder by Numbers
The Gift
O
Out of the Blue
Pan
Exodus: Gods and Kings
The Return of the Liv

In [91]:
for k, v in dict_test.items():
    if v[0] != 1 :
        print(k, v)

Hercules [2, 'Hercules', 'Hercules']
Twilight [2, 'Twilight', 'Twilight']
Home [3, 'Home', 'Home', 'Home']
Syriana [2, 'Syriana', 'Syriana']
Juno [2, 'Juno', 'Juno']
TotalRecall [2, 'Total Recall', 'Total Recall']
TheKarateKid [2, 'The Karate Kid', 'The Karate Kid']
OutoftheBlue [2, 'Out of the Blue', 'Out of the Blue']
HistoryoftheWorldPartI [2, 'History of the World: Part I', 'History of the World: Part I']
Crossroads [2, 'Crossroads', 'Crossroads']
TheIsland [2, 'The Island', 'The Island']
TheWatch [2, 'The Watch', 'The Watch']
Unbroken [2, 'Unbroken', 'Unbroken']
DeathataFuneral [2, 'Death at a Funeral', 'Death at a Funeral']
TheDeadZone [2, 'The Dead Zone            ', 'The Dead Zone']
KingKong [3, 'King Kong', 'King Kong', 'King Kong']
TheIllusionist [2, 'The Illusionist', 'The Illusionist']
Footloose [2, 'Footloose', 'Footloose']
Skyfall [2, 'Skyfall', 'Skyfall']
TheGreatGatsby [2, 'The Great Gatsby', 'The Great Gatsby']
MySoultoTake [2, 'My Soul to Take', 'My Soul to Take']
ANi